# Ungraded Lab: Hyperparameter tuning and model training with TFX

In this lab, you will be again doing hyperparameter tuning but this time, it will be within a [Tensorflow Extended (TFX)](https://www.tensorflow.org/tfx/) pipeline. 

We have already introduced some TFX components in Course 2 of this specialization related to data ingestion, validation, and transformation. In this notebook, you will get to work with two more which are related to model development and training: *Tuner* and *Trainer*.

<img src='https://www.tensorflow.org/tfx/guide/images/prog_trainer.png' alt='tfx pipeline'>
image source: https://www.tensorflow.org/tfx/guide

* The *Tuner* utilizes the [Keras Tuner](https://keras-team.github.io/keras-tuner/) API under the hood to tune your model's hyperparameters.
* You can get the best set of hyperparameters from the Tuner component and feed it into the *Trainer* component to optimize your model for training.

You will again be working with the [FashionMNIST](https://github.com/zalandoresearch/fashion-mnist) dataset and will feed it though the TFX pipeline up to the Trainer component.You will quickly review the earlier components from Course 2, then focus on the two new components introduced.

Let's begin!



## Setup

### Install TFX

You will first install [TFX](https://www.tensorflow.org/tfx), a framework for developing end-to-end machine learning pipelines.

In [3]:
!pip install tensorflow_datasets

  Using cached tensorflow_datasets-4.4.0-py3-none-any.whl (4.0 MB)
  Using cached promise-2.3-py3-none-any.whl
  Using cached importlib_resources-5.4.0-py3-none-any.whl (28 kB)


In [1]:
!pip install tfx==1.2

     |████████████████████████████████| 2.4 MB 1.7 MB/s eta 0:00:01
  Using cached packaging-20.9-py2.py3-none-any.whl (40 kB)
     |████████████████████████████████| 195.7 MB 14 kB/s  eta 0:00:01    |████████▋                       | 53.0 MB 2.8 MB/s eta 0:00:51     |███████████████████████████▏    | 166.0 MB 3.8 MB/s eta 0:00:08
     |████████████████████████████████| 54 kB 2.1 MB/s eta 0:00:011
     |████████████████████████████████| 1.2 MB 234 kB/s eta 0:00:01
     |████████████████████████████████| 20.3 MB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 406 kB 5.5 MB/s eta 0:00:01
  Using cached tensorflow_hub-0.12.0-py2.py3-none-any.whl (108 kB)
  Using cached tensorflow_serving_api-2.7.0-py2.py3-none-any.whl (37 kB)
     |████████████████████████████████| 1.5 MB 5.2 MB/s eta 0:00:01
  Using cached portpicker-1.5.0-py3-none-any.whl (14 kB)
     |████████████████████████████████| 189 kB 6.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 4.9 MB/s eta 0

*Note: In Google Colab, you need to restart the runtime at this point to finalize updating the packages you just installed. You can do so by clicking the `Restart Runtime` at the end of the output cell above (after installation), or by selecting `Runtime > Restart Runtime` in the Menu bar. **Please do not proceed to the next section without restarting.** You can also ignore the errors about version incompatibility of some of the bundled packages because we won't be using those in this notebook.*

### Imports

You will then import the packages you will need for this exercise.

In [4]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

import os
import pprint

from tfx.components import ImportExampleGen
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Transform
from tfx.components import Tuner
from tfx.components import Trainer

from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

## Download and prepare the dataset

As mentioned earlier, you will be using the Fashion MNIST dataset just like in the previous lab. This will allow you to compare the similarities and differences when using Keras Tuner as a standalone library and within an ML pipeline.

You will first need to setup the directories that you will use to store the dataset, as well as the pipeline artifacts and metadata store.

In [5]:
# Location of the pipeline metadata store
_pipeline_root = './pipeline/'

# Directory of the raw data files
_data_root = './data/fmnist'

# Temporary directory
tempdir = './tempdir'

In [6]:
# Create the dataset directory
!mkdir -p {_data_root}

# Create the TFX pipeline files directory
!mkdir {_pipeline_root}

You will now download FashionMNIST from [Tensorflow Datasets](https://www.tensorflow.org/datasets). The `with_info` flag will be set to `True` so you can display information about the dataset in the next cell (i.e. using `ds_info`).

In [7]:
# Download the dataset
ds, ds_info = tfds.load('fashion_mnist', data_dir=tempdir, with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/1 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/2 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/3 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/4 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/4 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Size...:   0%|          | 0/4 [00:00<?, ? MiB/s]

Dl Size...:   0%|          | 0/29 [00:00<?, ? MiB/s]

Dl Size...:   0%|          | 0/29 [00:00<?, ? MiB/s]

Dl Size...:   0%|          | 0/29 [00:00<?, ? MiB/s]

Dl Size...:   0%|          | 0/29 [00:00<?, ? MiB/s]

Extraction completed...:   0%|          | 0/1 [00:00<?, ? file/s]

Dl Size...:   0%|          | 0/29 [00:00<?, ? MiB/s]

Dl Size...:   0%|          | 0/29 [00:00<?, ? MiB/s]

Dl Size...:   0

Dataset fashion_mnist downloaded and prepared to ./tempdir/fashion_mnist/3.0.1. Subsequent calls will reuse this data.


In [8]:
# Display info about the dataset
print(ds_info)

tfds.core.DatasetInfo(
    name='fashion_mnist',
    full_name='fashion_mnist/3.0.1',
    description="""
    Fashion-MNIST is a dataset of Zalando's article images consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.
    """,
    homepage='https://github.com/zalandoresearch/fashion-mnist',
    data_path='./tempdir/fashion_mnist/3.0.1',
    download_size=29.45 MiB,
    dataset_size=36.42 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{DBLP:journals/corr/abs-1708-07747,
      author    = {Han Xiao and
                   K

You can review the downloaded files with the code below. For this lab, you will be using the *train* TFRecord so you will need to take note of its filename. You will not use the *test* TFRecord in this lab.

In [9]:
# Define the location of the train tfrecord downloaded via TFDS
tfds_data_path = f'{tempdir}/{ds_info.name}/{ds_info.version}'

# Display contents of the TFDS data directory
os.listdir(tfds_data_path)

['features.json',
 'fashion_mnist-test.tfrecord-00000-of-00001',
 'dataset_info.json',
 'label.labels.txt',
 'fashion_mnist-train.tfrecord-00000-of-00001']

You will then copy the train split from the downloaded data so it can be consumed by the ExampleGen component in the next step. This component requires that your files are in a directory without extra files (e.g. JSONs and TXT files).

In [10]:
# Define the train tfrecord filename
train_filename = 'fashion_mnist-train.tfrecord-00000-of-00001'

# Copy the train tfrecord into the data root folder
!cp {tfds_data_path}/{train_filename} {_data_root}

## TFX Pipeline

With the setup complete, you can now proceed to creating the pipeline. 

### Initialize the Interactive Context

You will start by initializing the [InteractiveContext](https://www.tensorflow.org/tfx/api_docs/python/tfx/orchestration/experimental/interactive/interactive_context/InteractiveContext) so you can run the components within this Colab environment. You can safely ignore the warning because you will just be using a local SQLite file for the metadata store.

In [11]:
# Initialize the InteractiveContext
context = InteractiveContext(pipeline_root=_pipeline_root)

### ExampleGen

You will start the pipeline by ingesting the TFRecord you set aside. The [ImportExampleGen](https://www.tensorflow.org/tfx/api_docs/python/tfx/components/ImportExampleGen) consumes TFRecords and you can specify splits as shown below. For this exercise, you will split the train tfrecord to use 80% for the train set, and the remaining 20% as eval/validation set.

In [12]:
# Specify 80/20 split for the train and eval set
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=2),
    ]))

# Ingest the data through ExampleGen
example_gen = ImportExampleGen(input_base=_data_root, output_config=output)

# Run the component
context.run(example_gen)

ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "./pipeline/ImportExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:33677636,xor_checksum:1640837915,sum_checksum:1640837915"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [13]:
# Print split names and URI
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] ./pipeline/ImportExampleGen/examples/1


### StatisticsGen

Next, you will compute the statistics of the dataset with the [StatisticsGen](https://www.tensorflow.org/tfx/guide/statsgen) component.

In [14]:
# Run StatisticsGen
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])

context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 16
        uri: "./pipeline/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### SchemaGen

You can then infer the dataset schema with [SchemaGen](https://www.tensorflow.org/tfx/guide/schemagen). This will be used to validate incoming data to ensure that it is formatted correctly.

In [15]:
# Run SchemaGen
schema_gen = SchemaGen(
      statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 18
        uri: "./pipeline/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [16]:
# Visualize the results
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'image',BYTES,required,,-
'label',INT,required,,-


### ExampleValidator

You can assume that the dataset is clean since we downloaded it from TFDS. But just to review, let's run it through [ExampleValidator](https://www.tensorflow.org/tfx/guide/exampleval) to detect if there are anomalies within the dataset.

In [17]:
# Run ExampleValidator
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 20
        uri: "./pipeline/ExampleValidator/anomalies/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
# Visualize the results. There should be no anomalies.
context.show(example_validator.outputs['anomalies'])

### Transform

Let's now use the [Transform](https://www.tensorflow.org/tfx/guide/transform) component to scale the image pixels and convert the data types to float. You will first define the transform module containing these operations before you run the component.

In [18]:
_transform_module_file = 'fmnist_transform.py'

In [19]:
%%writefile {_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

# Keys
_LABEL_KEY = 'label'
_IMAGE_KEY = 'image'


def _transformed_name(key):
    return key + '_xf'

def _image_parser(image_str):
    '''converts the images to a float tensor'''
    image = tf.image.decode_image(image_str, channels=1)
    image = tf.reshape(image, (28, 28, 1))
    image = tf.cast(image, tf.float32)
    return image


def _label_parser(label_id):
    '''converts the labels to a float tensor'''
    label = tf.cast(label_id, tf.float32)
    return label


def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
        inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
        Map from string feature key to transformed feature operations.
    """
    
    # Convert the raw image and labels to a float array
    with tf.device("/cpu:0"):
        outputs = {
            _transformed_name(_IMAGE_KEY):
                tf.map_fn(
                    _image_parser,
                    tf.squeeze(inputs[_IMAGE_KEY], axis=1),
                    dtype=tf.float32),
            _transformed_name(_LABEL_KEY):
                tf.map_fn(
                    _label_parser,
                    inputs[_LABEL_KEY],
                    dtype=tf.float32)
        }
    
    # scale the pixels from 0 to 1
    outputs[_transformed_name(_IMAGE_KEY)] = tft.scale_to_0_1(outputs[_transformed_name(_IMAGE_KEY)])
    
    return outputs

Writing fmnist_transform.py


You will run the component by passing in the examples, schema, and transform module file.

*Note: You can safely ignore the warnings and `udf_utils` related errors.*

In [20]:
# Ignore TF warning messages
tf.get_logger().setLevel('ERROR')

# Setup the Transform component
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_transform_module_file))

# Run the component
context.run(transform)

ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 5
        type_id: 22
        uri: "./pipeline/Transform/transform_graph/5"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 22
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 6
        type_id: 14
        uri: "./pipeline/Transform/transformed_examples/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 7
        type_id: 23
        uri: "./pipeline/Transform/updated_analyzer_cache/5"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 8
        type_id: 18
        uri: "./pipeline/Transform/pre_transform_schema/5"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , a

### Tuner

As the name suggests, the [Tuner](https://www.tensorflow.org/tfx/guide/tuner) component tunes the hyperparameters of your model. To use this, you will need to provide a *tuner module file* which contains a `tuner_fn()` function. In this function, you will mostly do the same steps as you did in the previous ungraded lab but with some key differences in handling the dataset. 

The Transform component earlier saved the transformed examples as TFRecords compressed in `.gz` format and you will need to load that into memory. Once loaded, you will need to create batches of features and labels so you can finally use it for hypertuning. This process is modularized in the `_input_fn()` below. 

Going back, the `tuner_fn()` function will return a `TunerFnResult` [namedtuple](https://docs.python.org/3/library/collections.html#collections.namedtuple) containing your `tuner` object and a set of arguments to pass to `tuner.search()` method. You will see these in action in the following cells. When reviewing the module file, we recommend viewing the `tuner_fn()` first before looking at the other auxiliary functions.

In [21]:
# Declare name of module file
_tuner_module_file = 'tuner.py'

In [22]:
%%writefile {_tuner_module_file}

# Define imports
from kerastuner.engine import base_tuner
import kerastuner as kt
from tensorflow import keras
from typing import NamedTuple, Dict, Text, Any, List
from tfx.components.trainer.fn_args_utils import FnArgs, DataAccessor
import tensorflow as tf
import tensorflow_transform as tft

# Declare namedtuple field names
TunerFnResult = NamedTuple('TunerFnResult', [('tuner', base_tuner.BaseTuner),
                                             ('fit_kwargs', Dict[Text, Any])])

# Label key
LABEL_KEY = 'label_xf'

# Callback for the search strategy
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)


def _gzip_reader_fn(filenames):
  '''Load compressed dataset
  
  Args:
    filenames - filenames of TFRecords to load

  Returns:
    TFRecordDataset loaded from the filenames
  '''

  # Load the dataset. Specify the compression type since it is saved as `.gz`
  return tf.data.TFRecordDataset(filenames, compression_type='GZIP')
  

def _input_fn(file_pattern,
              tf_transform_output,
              num_epochs=None,
              batch_size=32) -> tf.data.Dataset:
  '''Create batches of features and labels from TF Records

  Args:
    file_pattern - List of files or patterns of file paths containing Example records.
    tf_transform_output - transform output graph
    num_epochs - Integer specifying the number of times to read through the dataset. 
            If None, cycles through the dataset forever.
    batch_size - An int representing the number of records to combine in a single batch.

  Returns:
    A dataset of dict elements, (or a tuple of dict elements and label). 
    Each dict maps feature keys to Tensor or SparseTensor objects.
  '''

  # Get feature specification based on transform output
  transformed_feature_spec = (
      tf_transform_output.transformed_feature_spec().copy())
  
  # Create batches of features and labels
  dataset = tf.data.experimental.make_batched_features_dataset(
      file_pattern=file_pattern,
      batch_size=batch_size,
      features=transformed_feature_spec,
      reader=_gzip_reader_fn,
      num_epochs=num_epochs,
      label_key=LABEL_KEY)
  
  return dataset


def model_builder(hp):
  '''
  Builds the model and sets up the hyperparameters to tune.

  Args:
    hp - Keras tuner object

  Returns:
    model with hyperparameters to tune
  '''

  # Initialize the Sequential API and start stacking the layers
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28, 1)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu', name='dense_1'))

  # Add next layers
  model.add(keras.layers.Dropout(0.2))
  model.add(keras.layers.Dense(10, activation='softmax'))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])

  return model

def tuner_fn(fn_args: FnArgs) -> TunerFnResult:
  """Build the tuner using the KerasTuner API.
  Args:
    fn_args: Holds args as name/value pairs.

      - working_dir: working dir for tuning.
      - train_files: List of file paths containing training tf.Example data.
      - eval_files: List of file paths containing eval tf.Example data.
      - train_steps: number of train steps.
      - eval_steps: number of eval steps.
      - schema_path: optional schema of the input data.
      - transform_graph_path: optional transform graph produced by TFT.
  
  Returns:
    A namedtuple contains the following:
      - tuner: A BaseTuner that will be used for tuning.
      - fit_kwargs: Args to pass to tuner's run_trial function for fitting the
                    model , e.g., the training and validation dataset. Required
                    args depend on the above tuner's implementation.
  """

  # Define tuner search strategy
  tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory=fn_args.working_dir,
                     project_name='kt_hyperband')

  # Load transform output
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

  # Use _input_fn() to extract input features and labels from the train and val set
  train_set = _input_fn(fn_args.train_files[0], tf_transform_output)
  val_set = _input_fn(fn_args.eval_files[0], tf_transform_output)


  return TunerFnResult(
      tuner=tuner,
      fit_kwargs={ 
          "callbacks":[stop_early],
          'x': train_set,
          'validation_data': val_set,
          'steps_per_epoch': fn_args.train_steps,
          'validation_steps': fn_args.eval_steps
      }
  )

Writing tuner.py


With the module defined, you can now setup the Tuner component. You can see the description of each argument [here](https://www.tensorflow.org/tfx/api_docs/python/tfx/components/Tuner). 

Notice that we passed a `num_steps` argument to the train and eval args and this was used in the `steps_per_epoch` and `validation_steps` arguments in the tuner module above. This can be useful if you don't want to go through the entire dataset when tuning. For example, if you have 10GB of training data, it would be incredibly time consuming if you will iterate through it entirely just for one epoch and one set of hyperparameters. You can set the number of steps so your program will only go through a fraction of the dataset. 

You can compute for the total number of steps in one epoch by: `number of examples / batch size`. For this particular example, we have `48000 examples / 32 (default size)` which equals `1500` steps per epoch for the train set (compute val steps from 12000 examples). Since you passed `500` in the `num_steps` of the train args, this means that some examples will be skipped. This will likely result in lower accuracy readings but will save time in doing the hypertuning. Try modifying this value later and see if you arrive at the same set of hyperparameters.

In [23]:
from tfx.proto import trainer_pb2

# Setup the Tuner component
tuner = Tuner(
    module_file=_tuner_module_file,
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=500),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=100)
    )

In [24]:
# Run the component. This will take around 10 minutes to run.
context.run(tuner, enable_cache=False)

Epoch 1/2
500/500 [==============================] - ETA: 8:05 - loss: 2.4657 - accuracy: 0.09 - ETA: 1s - loss: 1.6536 - accuracy: 0.4330 - ETA: 1s - loss: 1.3216 - accuracy: 0.53 - ETA: 1s - loss: 1.1573 - accuracy: 0.58 - ETA: 1s - loss: 1.0622 - accuracy: 0.61 - ETA: 1s - loss: 0.9830 - accuracy: 0.64 - ETA: 1s - loss: 0.9392 - accuracy: 0.66 - ETA: 1s - loss: 0.9115 - accuracy: 0.67 - ETA: 1s - loss: 0.8790 - accuracy: 0.68 - ETA: 1s - loss: 0.8557 - accuracy: 0.69 - ETA: 1s - loss: 0.8328 - accuracy: 0.70 - ETA: 1s - loss: 0.8152 - accuracy: 0.71 - ETA: 1s - loss: 0.7988 - accuracy: 0.71 - ETA: 1s - loss: 0.7919 - accuracy: 0.72 - ETA: 1s - loss: 0.7837 - accuracy: 0.72 - ETA: 1s - loss: 0.7692 - accuracy: 0.73 - ETA: 1s - loss: 0.7566 - accuracy: 0.73 - ETA: 0s - loss: 0.7454 - accuracy: 0.73 - ETA: 0s - loss: 0.7344 - accuracy: 0.74 - ETA: 0s - loss: 0.7261 - accuracy: 0.74 - ETA: 0s - loss: 0.7165 - accuracy: 0.74 - ETA: 0s - loss: 0.7097 - accuracy: 0.75 - ETA: 0s - loss: 0.7

Epoch 1/2
500/500 [==============================] - ETA: 9:46 - loss: 2.4277 - accuracy: 0.18 - ETA: 2s - loss: 2.1673 - accuracy: 0.2102 - ETA: 2s - loss: 2.0296 - accuracy: 0.29 - ETA: 2s - loss: 1.9123 - accuracy: 0.36 - ETA: 2s - loss: 1.8106 - accuracy: 0.42 - ETA: 2s - loss: 1.7210 - accuracy: 0.46 - ETA: 2s - loss: 1.6651 - accuracy: 0.48 - ETA: 2s - loss: 1.5939 - accuracy: 0.51 - ETA: 2s - loss: 1.5434 - accuracy: 0.52 - ETA: 2s - loss: 1.4943 - accuracy: 0.53 - ETA: 2s - loss: 1.4549 - accuracy: 0.54 - ETA: 2s - loss: 1.4198 - accuracy: 0.55 - ETA: 2s - loss: 1.3846 - accuracy: 0.56 - ETA: 2s - loss: 1.3451 - accuracy: 0.57 - ETA: 2s - loss: 1.3124 - accuracy: 0.58 - ETA: 2s - loss: 1.2755 - accuracy: 0.60 - ETA: 1s - loss: 1.2528 - accuracy: 0.60 - ETA: 1s - loss: 1.2296 - accuracy: 0.61 - ETA: 1s - loss: 1.2105 - accuracy: 0.61 - ETA: 1s - loss: 1.1878 - accuracy: 0.62 - ETA: 1s - loss: 1.1652 - accuracy: 0.63 - ETA: 1s - loss: 1.1420 - accuracy: 0.63 - ETA: 1s - loss: 1.1

Epoch 1/2
500/500 [==============================] - ETA: 9:01 - loss: 2.3113 - accuracy: 0.12 - ETA: 3s - loss: 3.6172 - accuracy: 0.3403 - ETA: 4s - loss: 3.0621 - accuracy: 0.39 - ETA: 6s - loss: 2.7967 - accuracy: 0.41 - ETA: 5s - loss: 2.3051 - accuracy: 0.46 - ETA: 3s - loss: 1.6957 - accuracy: 0.56 - ETA: 3s - loss: 1.4103 - accuracy: 0.62 - ETA: 2s - loss: 1.2864 - accuracy: 0.64 - ETA: 2s - loss: 1.2016 - accuracy: 0.66 - ETA: 2s - loss: 1.1185 - accuracy: 0.67 - ETA: 2s - loss: 1.0805 - accuracy: 0.67 - ETA: 2s - loss: 1.0515 - accuracy: 0.68 - ETA: 2s - loss: 1.0309 - accuracy: 0.68 - ETA: 2s - loss: 1.0116 - accuracy: 0.68 - ETA: 1s - loss: 0.9866 - accuracy: 0.68 - ETA: 1s - loss: 0.9562 - accuracy: 0.69 - ETA: 1s - loss: 0.9417 - accuracy: 0.70 - ETA: 1s - loss: 0.9291 - accuracy: 0.70 - ETA: 1s - loss: 0.9164 - accuracy: 0.70 - ETA: 1s - loss: 0.9079 - accuracy: 0.70 - ETA: 1s - loss: 0.8894 - accuracy: 0.71 - ETA: 1s - loss: 0.8704 - accuracy: 0.71 - ETA: 1s - loss: 0.8

Epoch 1/2
500/500 [==============================] - ETA: 6:54 - loss: 2.3470 - accuracy: 0.06 - ETA: 1s - loss: 1.5712 - accuracy: 0.4781 - ETA: 1s - loss: 1.3007 - accuracy: 0.55 - ETA: 1s - loss: 1.1730 - accuracy: 0.60 - ETA: 1s - loss: 1.0846 - accuracy: 0.63 - ETA: 1s - loss: 1.0430 - accuracy: 0.64 - ETA: 1s - loss: 0.9909 - accuracy: 0.66 - ETA: 1s - loss: 0.9457 - accuracy: 0.67 - ETA: 1s - loss: 0.9143 - accuracy: 0.68 - ETA: 0s - loss: 0.8851 - accuracy: 0.70 - ETA: 0s - loss: 0.8586 - accuracy: 0.70 - ETA: 0s - loss: 0.8423 - accuracy: 0.71 - ETA: 0s - loss: 0.8204 - accuracy: 0.72 - ETA: 0s - loss: 0.8004 - accuracy: 0.72 - ETA: 0s - loss: 0.7874 - accuracy: 0.73 - ETA: 0s - loss: 0.7790 - accuracy: 0.73 - ETA: 0s - loss: 0.7663 - accuracy: 0.73 - ETA: 0s - loss: 0.7574 - accuracy: 0.74 - ETA: 0s - loss: 0.7485 - accuracy: 0.74 - ETA: 0s - loss: 0.7410 - accuracy: 0.74 - ETA: 0s - loss: 0.7376 - accuracy: 0.74 - ETA: 0s - loss: 0.7287 - accuracy: 0.74 - ETA: 0s - loss: 0.7

Epoch 1/2
500/500 [==============================] - ETA: 7:58 - loss: 2.3318 - accuracy: 0.15 - ETA: 34s - loss: 2.9324 - accuracy: 0.2344 - ETA: 31s - loss: 2.6786 - accuracy: 0.312 - ETA: 36s - loss: 2.4225 - accuracy: 0.375 - ETA: 18s - loss: 2.1949 - accuracy: 0.433 - ETA: 6s - loss: 1.5672 - accuracy: 0.558 - ETA: 4s - loss: 1.3308 - accuracy: 0.58 - ETA: 5s - loss: 1.3231 - accuracy: 0.58 - ETA: 4s - loss: 1.2508 - accuracy: 0.60 - ETA: 4s - loss: 1.1492 - accuracy: 0.62 - ETA: 3s - loss: 1.0644 - accuracy: 0.64 - ETA: 3s - loss: 0.9945 - accuracy: 0.66 - ETA: 2s - loss: 0.9467 - accuracy: 0.67 - ETA: 2s - loss: 0.9215 - accuracy: 0.68 - ETA: 2s - loss: 0.9033 - accuracy: 0.68 - ETA: 2s - loss: 0.8866 - accuracy: 0.68 - ETA: 2s - loss: 0.8682 - accuracy: 0.69 - ETA: 1s - loss: 0.8584 - accuracy: 0.69 - ETA: 1s - loss: 0.8419 - accuracy: 0.70 - ETA: 1s - loss: 0.8340 - accuracy: 0.70 - ETA: 1s - loss: 0.8243 - accuracy: 0.70 - ETA: 1s - loss: 0.8145 - accuracy: 0.70 - ETA: 1s - l

...

Epoch 1/10
500/500 [==============================] - ETA: 6:28 - loss: 2.7470 - accuracy: 0.09 - ETA: 1s - loss: 2.5334 - accuracy: 0.4258 - ETA: 1s - loss: 1.8318 - accuracy: 0.52 - ETA: 1s - loss: 1.5355 - accuracy: 0.57 - ETA: 1s - loss: 1.3811 - accuracy: 0.60 - ETA: 1s - loss: 1.2971 - accuracy: 0.61 - ETA: 1s - loss: 1.1834 - accuracy: 0.64 - ETA: 1s - loss: 1.1079 - accuracy: 0.65 - ETA: 1s - loss: 1.0597 - accuracy: 0.66 - ETA: 1s - loss: 1.0241 - accuracy: 0.66 - ETA: 1s - loss: 0.9972 - accuracy: 0.67 - ETA: 1s - loss: 0.9746 - accuracy: 0.68 - ETA: 1s - loss: 0.9543 - accuracy: 0.68 - ETA: 1s - loss: 0.9395 - accuracy: 0.68 - ETA: 1s - loss: 0.9137 - accuracy: 0.69 - ETA: 1s - loss: 0.8940 - accuracy: 0.69 - ETA: 1s - loss: 0.8802 - accuracy: 0.70 - ETA: 1s - loss: 0.8645 - accuracy: 0.70 - ETA: 1s - loss: 0.8592 - accuracy: 0.71 - ETA: 0s - loss: 0.8477 - accuracy: 0.71 - ETA: 0s - loss: 0.8340 - accuracy: 0.71 - ETA: 0s - loss: 0.8220 - accuracy: 0.71 - ETA: 0s - loss: 0.

Epoch 1/10
500/500 [==============================] - ETA: 7:37 - loss: 2.3393 - accuracy: 0.21 - ETA: 1s - loss: 2.3283 - accuracy: 0.4342 - ETA: 1s - loss: 1.6865 - accuracy: 0.54 - ETA: 1s - loss: 1.4121 - accuracy: 0.59 - ETA: 1s - loss: 1.3101 - accuracy: 0.61 - ETA: 1s - loss: 1.2436 - accuracy: 0.62 - ETA: 1s - loss: 1.1507 - accuracy: 0.63 - ETA: 1s - loss: 1.0772 - accuracy: 0.65 - ETA: 1s - loss: 1.0251 - accuracy: 0.66 - ETA: 1s - loss: 0.9863 - accuracy: 0.67 - ETA: 1s - loss: 0.9582 - accuracy: 0.68 - ETA: 1s - loss: 0.9326 - accuracy: 0.68 - ETA: 1s - loss: 0.9116 - accuracy: 0.69 - ETA: 0s - loss: 0.8871 - accuracy: 0.69 - ETA: 0s - loss: 0.8752 - accuracy: 0.70 - ETA: 0s - loss: 0.8685 - accuracy: 0.70 - ETA: 0s - loss: 0.8627 - accuracy: 0.70 - ETA: 0s - loss: 0.8598 - accuracy: 0.70 - ETA: 0s - loss: 0.8514 - accuracy: 0.70 - ETA: 0s - loss: 0.8436 - accuracy: 0.70 - ETA: 0s - loss: 0.8273 - accuracy: 0.71 - ETA: 0s - loss: 0.8196 - accuracy: 0.71 - ETA: 0s - loss: 0.

Epoch 1/10
500/500 [==============================] - ETA: 6:49 - loss: 2.4467 - accuracy: 0.12 - ETA: 1s - loss: 2.2248 - accuracy: 0.2012 - ETA: 1s - loss: 2.0566 - accuracy: 0.28 - ETA: 1s - loss: 1.9121 - accuracy: 0.37 - ETA: 1s - loss: 1.8100 - accuracy: 0.42 - ETA: 1s - loss: 1.7227 - accuracy: 0.46 - ETA: 1s - loss: 1.6485 - accuracy: 0.48 - ETA: 1s - loss: 1.5734 - accuracy: 0.51 - ETA: 1s - loss: 1.5222 - accuracy: 0.52 - ETA: 1s - loss: 1.4766 - accuracy: 0.53 - ETA: 1s - loss: 1.4497 - accuracy: 0.54 - ETA: 1s - loss: 1.4327 - accuracy: 0.54 - ETA: 1s - loss: 1.4059 - accuracy: 0.55 - ETA: 1s - loss: 1.3659 - accuracy: 0.56 - ETA: 1s - loss: 1.3301 - accuracy: 0.57 - ETA: 1s - loss: 1.2940 - accuracy: 0.58 - ETA: 1s - loss: 1.2668 - accuracy: 0.59 - ETA: 1s - loss: 1.2429 - accuracy: 0.60 - ETA: 1s - loss: 1.2252 - accuracy: 0.60 - ETA: 1s - loss: 1.2067 - accuracy: 0.61 - ETA: 0s - loss: 1.1850 - accuracy: 0.61 - ETA: 0s - loss: 1.1610 - accuracy: 0.62 - ETA: 0s - loss: 1.

Epoch 1/10
500/500 [==============================] - ETA: 9:31 - loss: 2.2418 - accuracy: 0.18 - ETA: 1s - loss: 1.9922 - accuracy: 0.3750 - ETA: 2s - loss: 1.8624 - accuracy: 0.40 - ETA: 3s - loss: 1.6599 - accuracy: 0.45 - ETA: 2s - loss: 1.3907 - accuracy: 0.53 - ETA: 1s - loss: 1.2277 - accuracy: 0.57 - ETA: 1s - loss: 1.1025 - accuracy: 0.60 - ETA: 1s - loss: 1.0399 - accuracy: 0.62 - ETA: 1s - loss: 1.0064 - accuracy: 0.63 - ETA: 1s - loss: 0.9619 - accuracy: 0.64 - ETA: 1s - loss: 0.9264 - accuracy: 0.66 - ETA: 0s - loss: 0.8976 - accuracy: 0.66 - ETA: 0s - loss: 0.8706 - accuracy: 0.67 - ETA: 0s - loss: 0.8598 - accuracy: 0.68 - ETA: 0s - loss: 0.8452 - accuracy: 0.68 - ETA: 0s - loss: 0.8385 - accuracy: 0.69 - ETA: 0s - loss: 0.8310 - accuracy: 0.69 - ETA: 0s - loss: 0.8216 - accuracy: 0.69 - ETA: 0s - loss: 0.8164 - accuracy: 0.69 - ETA: 0s - loss: 0.8065 - accuracy: 0.70 - ETA: 0s - loss: 0.7934 - accuracy: 0.70 - ETA: 0s - loss: 0.7816 - accuracy: 0.71 - ETA: 0s - loss: 0.

ExecutionResult(
    component_id: Tuner
    execution_id: 6
    outputs:
        best_hyperparameters: Channel(
            type_name: HyperParameters
            artifacts: [Artifact(artifact: id: 13
        type_id: 25
        uri: "./pipeline/Tuner/best_hyperparameters/6"
        custom_properties {
          key: "name"
          value {
            string_value: "best_hyperparameters"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Tuner"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 25
        name: "HyperParameters"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### Trainer

Like the Tuner component, the [Trainer](https://www.tensorflow.org/tfx/guide/trainer) component also requires a module file to setup the training process. It will look for a `run_fn()` function that defines and trains the model. The steps will look similar to the tuner module file:

* Define the model - You can get the results of the Tuner component through the `fn_args.hyperparameters` argument. You will see it passed into the `model_builder()` function below. If you didn't run `Tuner`, then you can just explicitly define the number of hidden units and learning rate.

* Load the train and validation sets - You have done this in the Tuner component. For this module, you will pass in a `num_epochs` value (10) to indicate how many batches will be prepared. You can opt not to do this and pass a `num_steps` value as before.

* Setup and train the model - This will look very familiar if you're already used to the [Keras Models Training API](https://keras.io/api/models/model_training_apis/). You can pass in callbacks like the [TensorBoard callback](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard) so you can visualize the results later.

* Save the model - This is needed so you can analyze and serve your model. You will get to do this in later parts of the course and specialization.

In [25]:
# Declare trainer module file
_trainer_module_file = 'trainer.py'

In [26]:
%%writefile {_trainer_module_file}

from tensorflow import keras
from typing import NamedTuple, Dict, Text, Any, List
from tfx.components.trainer.fn_args_utils import FnArgs, DataAccessor
import tensorflow as tf
import tensorflow_transform as tft

# Define the label key
LABEL_KEY = 'label_xf'

def _gzip_reader_fn(filenames):
  '''Load compressed dataset
  
  Args:
    filenames - filenames of TFRecords to load

  Returns:
    TFRecordDataset loaded from the filenames
  '''

  # Load the dataset. Specify the compression type since it is saved as `.gz`
  return tf.data.TFRecordDataset(filenames, compression_type='GZIP')
  

def _input_fn(file_pattern,
              tf_transform_output,
              num_epochs=None,
              batch_size=32) -> tf.data.Dataset:
  '''Create batches of features and labels from TF Records

  Args:
    file_pattern - List of files or patterns of file paths containing Example records.
    tf_transform_output - transform output graph
    num_epochs - Integer specifying the number of times to read through the dataset. 
            If None, cycles through the dataset forever.
    batch_size - An int representing the number of records to combine in a single batch.

  Returns:
    A dataset of dict elements, (or a tuple of dict elements and label). 
    Each dict maps feature keys to Tensor or SparseTensor objects.
  '''
  transformed_feature_spec = (
      tf_transform_output.transformed_feature_spec().copy())
  
  dataset = tf.data.experimental.make_batched_features_dataset(
      file_pattern=file_pattern,
      batch_size=batch_size,
      features=transformed_feature_spec,
      reader=_gzip_reader_fn,
      num_epochs=num_epochs,
      label_key=LABEL_KEY)
  
  return dataset


def model_builder(hp):
  '''
  Builds the model and sets up the hyperparameters to tune.

  Args:
    hp - Keras tuner object

  Returns:
    model with hyperparameters to tune
  '''

  # Initialize the Sequential API and start stacking the layers
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28, 1)))

  # Get the number of units from the Tuner results
  hp_units = hp.get('units')
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))

  # Add next layers
  model.add(keras.layers.Dropout(0.2))
  model.add(keras.layers.Dense(10, activation='softmax'))

  # Get the learning rate from the Tuner results
  hp_learning_rate = hp.get('learning_rate')

  # Setup model for training
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])

  # Print the model summary
  model.summary()
  
  return model


def run_fn(fn_args: FnArgs) -> None:
  """Defines and trains the model.
  Args:
    fn_args: Holds args as name/value pairs. Refer here for the complete attributes: 
    https://www.tensorflow.org/tfx/api_docs/python/tfx/components/trainer/fn_args_utils/FnArgs#attributes
  """

  # Callback for TensorBoard
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=fn_args.model_run_dir, update_freq='batch')
  
  # Load transform output
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
  
  # Create batches of data good for 10 epochs
  train_set = _input_fn(fn_args.train_files[0], tf_transform_output, 10)
  val_set = _input_fn(fn_args.eval_files[0], tf_transform_output, 10)

  # Load best hyperparameters
  hp = fn_args.hyperparameters.get('values')

  # Build the model
  model = model_builder(hp)

  # Train the model
  model.fit(
      x=train_set,
      validation_data=val_set,
      callbacks=[tensorboard_callback]
      )
  
  # Save the model
  model.save(fn_args.serving_model_dir, save_format='tf')

Writing trainer.py


You can pass the output of the `Tuner` component to the `Trainer` by filling the `hyperparameters` argument with the `Tuner` output. This is indicated by the `tuner.outputs['best_hyperparameters']` below. You can see the definition of the other arguments [here](https://www.tensorflow.org/tfx/api_docs/python/tfx/components/Trainer).

In [27]:
# Setup the Trainer component
trainer = Trainer(
    module_file=_trainer_module_file,
    examples=transform.outputs['transformed_examples'],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval']))

Take note that when re-training your model, you don't always have to retune your hyperparameters. Once you have a set that you think performs well, you can just import it with the ImporterNode as shown in the [official docs](https://www.tensorflow.org/tfx/guide/tuner):

```
hparams_importer = ImporterNode(
    instance_name='import_hparams',
    # This can be Tuner's output file or manually edited file. The file contains
    # text format of hyperparameters (kerastuner.HyperParameters.get_config())
    source_uri='path/to/best_hyperparameters.txt',
    artifact_type=HyperParameters)

trainer = Trainer(
    ...
    # An alternative is directly use the tuned hyperparameters in Trainer's user
    # module code and set hyperparameters to None here.
    hyperparameters = hparams_importer.outputs['result'])
```

In [28]:
# Run the component
context.run(trainer, enable_cache=False)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 96)                75360     
_________________________________________________________________
dropout_1 (Dropout)          (None, 96)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                970       
Total params: 76,330
Trainable params: 76,330
Non-trainable params: 0
_________________________________________________________________
14993/14993 [==============================]- accuracy: 0.15 - 1s 122ms/step - loss: 2.2972 - accuracy: 0.156 - 1s 86ms/step - loss: 2.2641 - accuracy: 0.166 - 1s 13ms/step - loss: 1.6769 - accuracy: 0.41 - 1s 8ms/step - loss: 1.3913 - accuracy: 0.

ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 14
        type_id: 27
        uri: "./pipeline/Trainer/model/7"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 27
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 15
        type_id: 28
        uri: "./pipeline/Trainer/model_run/7"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 28
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

Your model should now be saved in your pipeline directory and you can navigate through it as shown below. The file is saved as `saved_model.pb`.

In [29]:
# Get artifact uri of trainer model output
model_artifact_dir = trainer.outputs['model'].get()[0].uri

# List subdirectories artifact uri
print(f'contents of model artifact directory:{os.listdir(model_artifact_dir)}')

# Define the model directory
model_dir = os.path.join(model_artifact_dir, 'Format-Serving')

# List contents of model directory
print(f'contents of model directory: {os.listdir(model_dir)}')

contents of model artifact directory:['Format-Serving']
contents of model directory: ['keras_metadata.pb', 'variables', 'saved_model.pb', 'assets']


You can also visualize the training results by loading the logs saved by the Tensorboard callback.

In [31]:
model_run_artifact_dir = trainer.outputs['model_run'].get()[0].uri

%load_ext tensorboard
%tensorboard --logdir {model_run_artifact_dir}

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 29617), started 0:00:31 ago. (Use '!kill 29617' to kill it.)

***Congratulations! You have now created an ML pipeline that includes hyperparameter tuning and model training. You will know more about the next components in future lessons but in the next section, you will first learn about a framework for automatically building ML pipelines: AutoML. Enjoy the rest of the course!***